In [314]:
import pandas as pd
import numpy as np
import spotipy
import spotipy.util as util
from spotipy import oauth2
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import os

# Data Preperation

### In this notebook, two dataset will be created: "df_top" and "df_liked"

Both of these dataset will include first 100 songs of popular playlists by Spotify as "not favorite".
df_top will also include user's top 50 songs 
df_liked will include user's liked songs


In [315]:
# Putting spotify credentials
os.environ['SPOTIPY_CLIENT_ID'] ='3a296280058f458d881e4caf58ff630f'
os.environ['SPOTIPY_CLIENT_SECRET'] ='2a815b9a6d974667be04307685c3215f'
os.environ['SPOTIPY_REDIRECT_URI'] ='http://localhost:7777/callback'

username = 'pvinqu86bwvr33t8tydnxdmh9'

In [316]:
scope = "user-read-private, streaming, playlist-read-collaborative, user-library-read,user-read-email, playlist-read-private, user-read-playback-position, user-library-modify, user-follow-read, user-read-currently-playing, user-follow-modify, playlist-modify-private, user-modify-playback-state, playlist-modify-public, app-remote-control, user-top-read, user-read-playback-state, user-read-recently-played,"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

## Creating a dataset of playlist that contains first 100 songs of popular Spotify's playlist

##### Note: You can use this code to get all the 1388 playlist IDs by Spotify. But this will create too large dataset when I include audio features.

In [317]:
# # Getting playlist IDs from each of Spotify's playlists
# playlists = sp.user_playlists('spotify')
# spotify_playlist_ids = []
# while playlists:
#     for i, playlist in enumerate(playlists['items']):
#         spotify_playlist_ids.append(playlist['uri'][-22:])
#     if playlists['next']:
#         playlists = sp.next(playlists)
#     else:
#         playlists = None

##### Use this code to get access to certain popular playlists like
"Today's Top Hits", 
"Rap Caviar", 
"New Music Friday", 
"Are & Be", 
"Mint", 
"just hits", 
"soft pop hits", 


In [318]:
popular_playlist_ids = ['37i9dQZF1DXcBWIGoYBM5M', '37i9dQZF1DX0XUsuxWHRQd', 
                        '37i9dQZF1DX4JAvHpjipBk', '37i9dQZF1DX4SBhb3fqCJd', 
                        '37i9dQZF1DX4dyzvuaRJ0n', '37i9dQZF1DXcRXFNfZr7Tp', 
                        '37i9dQZF1DWTwnEm1IYyoj', '37i9dQZF1DX5hR0J49CmXC',
                       '37i9dQZF1DXan38dNVDdl4', '37i9dQZF1DX4o1oenSJRJd', '37i9dQZF1DXbTxeAdrVG2l',
                       '37i9dQZF1DX4UtSsGT1Sbe', '37i9dQZF1DWTJ7xPn4vNaz', '37i9dQZF1DWY4xHQp97fN6', 
                       '37i9dQZF1DWVA1Gq4XHa6U', '37i9dQZF1DX2RxBh64BHjQ', '37i9dQZF1DX2A29LI7xHn1',
                       '37i9dQZF1DWX3387IZmjNa', '37i9dQZF1DWUVpAXiEPK8P', '37i9dQZF1DX0Tkc6ltcBfU']



In [298]:
# Function that gets the first 100 track IDs from each playlist
def getTrackFromPlaylist(playlist_id):
    playlist = sp.user_playlist('spotify', playlist_id)
    for item in playlist['tracks']['items'][:100]:
        try: 
            track = item['track']
            ids.append(track['id'])
        except:
            pass
    return ids



In [299]:
# Getting track ids given playlist ids
ids = []
for x in popular_playlist_ids:
    getTrackFromPlaylist(x)
    

In [300]:
#Creating a function that gets audio features of each track given the track id
def getAudioFeatures(track_id):
    song = sp.track(track_id)
    audio_features = sp.audio_features(track_id)
    
    track_id = track_id
    popularity = song['popularity']
    acousticness = audio_features[0]['acousticness']
    danceability = audio_features[0]['danceability']
    energy = audio_features[0]['energy']
    instrumentalness = audio_features[0]['instrumentalness']
    liveness = audio_features[0]['liveness']
    loudness = audio_features[0]['loudness']
    speechiness = audio_features[0]['speechiness']
    tempo = audio_features[0]['tempo']
    time_signature = audio_features[0]['time_signature']

    track = [track_id, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    return track

In [301]:
# Getting audio features given track ids
tracks = []
for i in range(len(ids)):
    try:  
        track = getAudioFeatures(ids[i])
        tracks.append(track)
    except:
        pass

In [302]:
# Creating a dataset of spotify tracks

df = pd.DataFrame(tracks, columns = ['track_id', 'popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df.shape


(1557, 11)

In [303]:
# Dropping duplicated songs
df = df.drop_duplicates(subset=['track_id'])
df.shape

(1348, 11)

## Creating a dataset of my TOP songs with their audio features

In [304]:
# Getting IDs of user's favorite tracks
top_tracks = sp.current_user_top_tracks(limit = 100)

fav_id = []
for i, items in enumerate(top_tracks['items']):
    fav_id.append(items['id'])

In [305]:
# Getting track features for each song in favorite song dataframe
fav_tracks = []
for track in fav_id:
    try:  
        track = getAudioFeatures(track)
        fav_tracks.append(track)
    except:
        pass

In [306]:
# Creating a dataset df_fav


df_fav = pd.DataFrame(fav_tracks, columns = ['track_id', 'popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df_fav.shape


(50, 11)

In [307]:
# Checking duplicate values
df_fav['track_id'].value_counts().head()

3FskQrDXcY24ur2fCvz35O    1
3d7NcI1X25bL2icPmfRbBs    1
1u2hPlWD5rlCzDa0X6zHaf    1
3WmBWBfKHdKuWIsCX0rRFd    1
1nf3SiU98SfKVJhkuAx9uj    1
Name: track_id, dtype: int64

## Creating a dataset of my LIKED songs with their audio features.

In [308]:
# Getting IDs of user's liked tracks
liked_tracks = sp.current_user_saved_tracks(limit = 50)




In [309]:
liked_id =[]

for i, items in enumerate(liked_tracks['items']):
    liked_id.append(items['track']['id'])

In [310]:
# Getting track features for each song in favorite song dataframe
liked_tracks = []
for track in liked_id:
    try:  
        track = getAudioFeatures(track)
        liked_tracks.append(track)
    except:
        pass


In [311]:
# Creating a dataset df_fav
df_liked = pd.DataFrame(liked_tracks, columns = ['track_id', 'popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df_liked.shape

(50, 11)

## Final datasets: Creating two datasets and creating a column "favorite" for each dataset

### Dataset 1: Top songs

In [312]:
#Creating a favorite column 
df_fav['favorite'] = 1
df['favorite'] = 0 

# Combining two datasets
data_top = pd.concat([df, df_fav])

#Dropping duplicate values
data_top = data_top.drop_duplicates(subset = 'track_id', keep = 'last')

data_top['favorite'].value_counts()

# Exporting the data
data_top.to_csv('my_spotify_data_with_top_tracks.csv', index=False)

### Dataset 2: Liked songs

In [313]:
#Creating a favorite column 
df_liked['favorite'] = 1
#df['favorite'] = 0 

# Combining two datasets
data_liked = pd.concat([df, df_liked])

#Dropping duplicate values
data_liked = data_liked.drop_duplicates(subset = 'track_id', keep = 'last')

data_liked['favorite'].value_counts()

# Exporting the data
data_liked.to_csv('my_spotify_data_with_liked_tracks.csv', index=False)